In [ ]:
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# 使每个Cell同时可输出多个语句的值
%config ZMQInteractiveShell.ast_node_interactivity='all'

## 数据转换

In [ ]:
# numpy array
np_data = np.arange(6).reshape((2, 3))
np_data

# numpy array --> torch tensor
torch_data = torch.from_numpy(np_data)
torch_data

# torch tensor --> numpy array
tensor2array = torch_data.numpy()
tensor2array

## 使用torch.cat拼接数据

In [ ]:
n_data = torch.ones(100, 2)         # 数据的基本形态
x0 = torch.normal(2*n_data, 1)      # x0.shape=(100, 2)
x1 = torch.normal(-2*n_data, 1)     # x1.shape=(100, 2)

# 注意 x, y 数据的数据形式是一定要像下面一样 (torch.cat 是在合并数据) x.shape=(200, 2)
x = torch.cat((x0, x1), dim=0).type(torch.FloatTensor)  # FloatTensor = 32-bit floating

## 在新的维度长连接tensors序列. 输出的张量维度比输入的张量多一维
stack(tensors, dim=0, out=None)
   + tensors: 要连接的张量序列
   + dim: 新增的维度(张量序列在这个维度上进行连接)

In [ ]:
t1 = np.array([[1, 2], [3, 4], [5, 6]])
t1 = torch.from_numpy(t1)
t1

t2 = np.array([[11, 12], [13, 14], [15, 16]])
t2 = torch.from_numpy(t2)
t2

t1.shape
t2.shape
t = torch.stack((t1, t2), dim=0)
t.shape
t

t = torch.stack([t1, t2], dim=1)
t.shape
t

# dim最大只能取2, 因为t1/t2是2维的,stack增加了一个维度(3维) --对应dim=2
t = torch.stack((t1, t2), dim=2)
t.shape
t


In [ ]:
torch.stack?

## 随机生成数据


### 从序列中随机选择
从序列中随机选择(有放回)若干元素组成一个新序列. --np.random.choice
+ eg: np.random.choice([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 3)

In [ ]:
np.random.choice([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 3)

### 生成指定维度的随机数

+ eg: np.random.rand(3, 2)
    * 生成指定维度的随机数,元素取值范围为[0, 1)
    
+ np.random.randint(0, 10, size=(3,2))
    * 从指定区间范围中随机生成一个指定shape的array

In [ ]:
randomArrayFloat = np.random.rand(3, 2)  # 生成的是float64类型的
randomArrayFloat
randomArrayFloat.dtype
randomArrayFloat = randomArrayFloat.astype('float32') # 转换成float32类型的

# 从[0, 10)中随机生成一个数
randomInt = np.random.randint(0, 10)
randomInt

# 从[0, 10)中随机生成一个指定shape的array
randomIntArray = np.random.randint(0, 10, size=(3,2))
randomIntArray

### 按指定分布采样数据

In [ ]:
# 从均匀分布中抽取样本。-- np.random.uniform
uniformData = np.random.uniform(1, 2, size=50) #从[1,2]中使用均匀分布抽取size个样本
np.mean(uniformData)
np.std(uniformData)

### 数据维度的说明

In [ ]:
data1 = np.linspace(0, 5, 5, dtype=np.float32, endpoint=False)
data2 = np.linspace(5, 10, 5, dtype=np.float32, endpoint=False)
data1 = np.row_stack((data1, data2))
data1.shape

data2 = data1[np.newaxis, :, np.newaxis, :]  # 新增了两个维度
data2.shape
data2

'''
输出:
(2, 5)

(1, 2, 1, 5)

array([[[[0., 1., 2., 3., 4.]],

        [[5., 6., 7., 8., 9.]]]], dtype=float32)
'''

## RNN

### RNN 做回归任务

In [ ]:
# 设置超参数
TIME_STEP = 10      # 每个batch中的样本数
INPUT_SIZE = 1      # 输入RNN的单个样本的特征数
LR = 0.02 # learning rate
BIDIRECTIONAL= False #是否为双向RNN
HIDDEN_SIZE = 32
EPOCH = 100

class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()

        self.rnn = nn.RNN(
            input_size=INPUT_SIZE,
            hidden_size=HIDDEN_SIZE,     # rnn 单隐层神经元数目
            num_layers=1,                # rnn 隐层数目
            batch_first=True,            # input & output将batchsize放在第一个维度(batch, time_step, input_size) 
            bidirectional=BIDIRECTIONAL  #是否为双向RNN
        )
        in_features = HIDDEN_SIZE
        if BIDIRECTIONAL: # 使用双向RNN时Linear的输入特征数要翻倍
            in_features *= 2
        self.out = nn.Linear(in_features, 1)

    def forward(self, x, h_state):
        # batch_first=True时各参数的shape
        # x.shape (batch, time_step, input_size)
        # h_state.shpae (n_layers * num_directions, batch, hidden_size) --双向RNN时num_directions为2,否则num_direction为1
        # r_out.shpae (batch, time_step, hidden_size*num_directions)
        r_out, h_state = self.rnn(x, h_state)

        #print(r_out.shape)
        outs = []    # 保存当前batch所有样本的预测输出
        for time_step in range(r_out.size(1)):    # 计算当前batch每一个样本的预测输出
            # r_out[:, time_step, :]获取的是每个样本的所有batch的最后一个隐层的参数
            # self.out(r_out[:, time_step, :])则得到当前送入RNN的所有batch样本的预测值 shape=(batch, out_features)
            outs.append(self.out(r_out[:, time_step, :])) 
        return torch.stack(outs, dim=1), h_state #torch.stack(outs, dim=1)得到shape=(batch, time_step, out_features)形状的结果作为下一个的输入


    
# 开始训练
rnn = RNN()
print(rnn)

optimizer = torch.optim.Adam(rnn.parameters(), lr=LR)   # rnn参数优化器
loss_func = nn.MSELoss()

plt.figure(1, figsize=(12, 5))
plt.ion() # 为了能进行连续绘图

h_state = None      # 初始化隐层状态
for step in range(EPOCH):
    # 为了演示一次向RNN送入多个batch数据的情况,手动构造两批样本点, 每次送入两个patch到RNN
    start, end = step * np.pi, (step+1)*np.pi   
    steps = np.linspace(start, end, TIME_STEP, dtype=np.float32, endpoint=False)  # float32 for converting torch FloatTensor
    x_np = np.sin(steps)
    y_np = np.cos(steps)
    x = torch.from_numpy(x_np[np.newaxis, :, np.newaxis])    # shape (batch, time_step, input_size)
    y = torch.from_numpy(y_np[np.newaxis, :, np.newaxis])
    
    x_np2 = np.sin(steps)
    y_np2 = (np.cos(steps) + np.random.rand(len(steps))/100).astype('float32') #加入扰动
    x2 = torch.from_numpy(x_np2[np.newaxis, :, np.newaxis])
    y2 = torch.from_numpy(y_np2[np.newaxis, :, np.newaxis])
    
    # 组合两个patch一次送入RNN
    x = torch.cat((x, x2), dim=0)
    y = torch.cat((y, y2), dim=0)

    # 计算预测值, 计算误差, 进行误差反向传播
    prediction, h_state = rnn(x, h_state)   # rnn output
    h_state = h_state.data    # !! next step is important !! repack the hidden state, break the connection from last iteration
    loss = loss_func(prediction, y)         # calculate loss
    optimizer.zero_grad()                   # clear gradients for this training step
    loss.backward()                         # backpropagation, compute gradients
    optimizer.step()                        # apply gradients
    

    # plotting
    plt.plot(steps, y_np.flatten(), 'r-')
    np_prediction = prediction.data.numpy().flatten()
    plt.plot(steps, (np_prediction[0:len(steps)]+np_prediction[len(steps):2*len(steps)])/2, 'b-')
    plt.draw() 
    plt.pause(0.05)
plt.ioff()
plt.show()

In [ ]:
np.random.choice([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 3)